In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import precision_score
import pandas as pd
import seaborn as sns
from statsmodels import api as sm
from matplotlib import pyplot as plt
from pandas.plotting import register_matplotlib_converters
import os
import numpy as np
import math
import validate
from sklearn.preprocessing import StandardScaler
import PR_curve
import librosa as lb
import os
import matplotlib.pyplot as plt
import glob
import wave
import IPython
from IPython.display import display
import datetime

In [2]:
data_dir='csv_label'
fixed_data_dir='fixed_csv_label_ver2'
df = pd.read_csv( os.path.join(data_dir, 'learning_data_10.csv') )
fixed_df = pd.read_csv( os.path.join(fixed_data_dir, 'learning_data_10.csv') )
df

,original,label,time,day
0,53.4,0,15,0
1,52.8,0,15,0
2,52.8,0,15,0
3,53.2,0,15,0
4,53.6,0,15,0
...,...,...,...,...
3995345,45.9,0,5,9
3995346,45.6,0,5,9
3995347,45.8,0,5,9
3995348,45.7,0,5,9


In [3]:
def calc_leq(df, unit):
    df.reset_index(inplace=True, drop=True)
    for i in range(int(len(df)/unit)+1):
        hour_df=df.loc[i*unit:(i+1)*unit, 'original'].copy()
        N=len(hour_df)
        Leq=10*np.log10(np.sum(np.power(10, hour_df/10)))-10*np.log10(N)
        df.loc[i*unit:(i+1)*unit, 'leq']=Leq
    return df

def figure(df_test):
    test_plot=df_test['original'].values
    num_ax=math.ceil(len(test_plot)/18000)
    
    label=df_test['label'].values*100
    label_index=range(len(label))

    time_unit=60
    time=[t*0.2/60 for t in range(len(df_test))]
    plt.rcParams["font.size"] = 30

    fig, ax=plt.subplots(num_ax, 1, figsize=(35, 15*num_ax))
    for i in range(num_ax):
        ax[i].plot(time, test_plot, '-k',linewidth = 2)
        ax[i].fill_between(time, label, facecolor='lime' )
        ax[i].set_ylim(0, 90)
        ax[i].set_xlim(i*time_unit, time_unit*(i+1))
        ax[i].set_xticks(np.arange(60*i, 60*(i+1)))
        ax[i].xaxis.set_tick_params(rotation=70)
    plt.show()

In [4]:
for x in set(df['day']):
    test_day=[x]
    df_test=df[df['day'].isin(test_day)].reset_index(drop=True)
    fixed_test=fixed_df[fixed_df['day'].isin(test_day)].reset_index(drop=True)

    added_test=fixed_test.copy()
    added_test.loc[(df_test['label']==1)&(fixed_test['label']==1), 'label']=0

    added_test=calc_leq(added_test, 18000)

    #     再現率
    df_anorm=[]
    search= 1 if added_test.loc[0, 'label']==0 else 0
    for num in range(len(added_test)):
        if num!=len(added_test)-1:
            if search==1 and added_test.loc[num, 'label']==search:
                start=num
                search=0
            elif search==0 and added_test.loc[num, 'label']==search:
                stop=num-1
                anorm_range=added_test.loc[start:stop].copy()
                df_anorm.append(anorm_range)
                search=1

        elif search==0:
            if added_test.loc[num, 'label']==search:
                stop=num-1
            else:
                stop=num
            anorm_range=added_test.loc[start:stop].copy()
            df_anorm.append(anorm_range)
            search=1

    num=len(df_anorm)
    print(f'{x}日目に追加した異常部分の数：{num}') 
    print(f'-------------------------------------------') 
    count=0
    for i in range(num):
        tmp=df_anorm[i]
        over_num=len(tmp[tmp['original']>=tmp['leq']])
        if over_num==0:
            count+=1
            start=tmp.index[0]*0.2
            stop=tmp.index[-1]*0.2
            print(f'{count}個目の埋もれた異常部分')
            print(f'\t開始：{int(start//60)}分{start%60}秒 / {str(datetime.timedelta(seconds=start))}')
            print(f'\t終了：{int(stop//60)}分{stop%60}秒 / {str(datetime.timedelta(seconds=stop))}')
    print(f'-------------------------------------------') 
    print(f'埋もれた異常音の数：{count}')
    print('===========================================================================\n\n')

0日目に追加した異常部分の数：109
-------------------------------------------
1個目の埋もれた異常部分
	開始：347分15.200000000000728秒 / 5:47:15.200000
	終了：347分22.600000000002183秒 / 5:47:22.600000
2個目の埋もれた異常部分
	開始：378分10.600000000002183秒 / 6:18:10.600000
	終了：378分16.600000000002183秒 / 6:18:16.600000
3個目の埋もれた異常部分
	開始：810分57.200000000004366秒 / 13:30:57.200000
	終了：810分57.400000000001455秒 / 13:30:57.400000
4個目の埋もれた異常部分
	開始：886分54.200000000004366秒 / 14:46:54.200000
	終了：886分57.400000000001455秒 / 14:46:57.400000
5個目の埋もれた異常部分
	開始：939分0.6000000000058208秒 / 15:39:00.600000
	終了：939分8.60000000000582秒 / 15:39:08.600000
6個目の埋もれた異常部分
	開始：1001分1.6000000000058208秒 / 16:41:01.600000
	終了：1001分11.60000000000582秒 / 16:41:11.600000
7個目の埋もれた異常部分
	開始：1016分5.600000000005821秒 / 16:56:05.600000
	終了：1016分15.60000000000582秒 / 16:56:15.600000
8個目の埋もれた異常部分
	開始：1036分43.60000000000582秒 / 17:16:43.600000
	終了：1036分52.400000000001455秒 / 17:16:52.400000
9個目の埋もれた異常部分
	開始：1038分19.60000000000582秒 / 17:18:19.600000
	終了：1038分52.60000000000582秒 / 17:18:52.600

4日目に追加した異常部分の数：60
-------------------------------------------
1個目の埋もれた異常部分
	開始：43分59.600000000000364秒 / 0:43:59.600000
	終了：44分2.800000000000182秒 / 0:44:02.800000
2個目の埋もれた異常部分
	開始：47分37.20000000000027秒 / 0:47:37.200000
	終了：47分42.40000000000009秒 / 0:47:42.400000
3個目の埋もれた異常部分
	開始：79分46.19999999999982秒 / 1:19:46.200000
	終了：79分51.400000000000546秒 / 1:19:51.400000
4個目の埋もれた異常部分
	開始：84分37.600000000000364秒 / 1:24:37.600000
	終了：84分39.80000000000018秒 / 1:24:39.800000
5個目の埋もれた異常部分
	開始：257分13.600000000000364秒 / 4:17:13.600000
	終了：257分18.80000000000109秒 / 4:17:18.800000
6個目の埋もれた異常部分
	開始：421分57.60000000000218秒 / 7:01:57.600000
	終了：422分4.80000000000291秒 / 7:02:04.800000
7個目の埋もれた異常部分
	開始：425分20.200000000000728秒 / 7:05:20.200000
	終了：425分25.80000000000291秒 / 7:05:25.800000
8個目の埋もれた異常部分
	開始：471分19.600000000002183秒 / 7:51:19.600000
	終了：471分25.80000000000291秒 / 7:51:25.800000
9個目の埋もれた異常部分
	開始：471分28.200000000000728秒 / 7:51:28.200000
	終了：471分33.400000000001455秒 / 7:51:33.400000
10個目の埋もれた異常部分
	開始：608分17.59999